In [330]:
import pandas as pd
import plotly.express as px
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## Análise Inicial

In [331]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [332]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [333]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


Podemos perceber que algumas colunas possuem valores nulos, iremos analisá-las melhor depois!

Além disso, irei doprar algumas colunas que não serão importantes, tanto na análise, quanto, no modelo preditivo.

In [334]:
train.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)

ids = test['PassengerId']
test.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)

train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


### Sobrevivência

In [335]:
fig = px.histogram(train, x='Survived')
fig.update_layout(xaxis_type='category')
fig.show()

### Relação Sobrevivência x Colunas

#### Coluna 'Sex'

In [336]:
fig = px.histogram(train, x='Sex', color='Survived', barmode='group')
fig.show()

As mulheres sobreviveram mais que os homens, muito provavelmente, pois elas eram preferência no processo de evacuação.

#### Coluna 'Pclass'

In [337]:
fig = px.histogram(train, x='Pclass', color='Survived', barmode='group')
fig.show()

Em relação às classes, é possível perceber que 3ª foi a mais prejudicada durante o desastre, provavelmente, por não serem prioridade durante a evacuação, além, estarem em locais não muito acessíveis às areas de evacuação e botes salva-vidas. 

#### Coluna 'SibSp'

In [338]:
fig = px.histogram(train, x='SibSp', color='Survived', barmode='group')
fig.show()

Proporcionalmente pessoas com 1 ou 2 irmãos/cônjuges sobreviveram mais.

In [339]:
fig = px.histogram(train, x='Parch', color='Survived', barmode='group')
fig.show()

Proporcionalmente pessoas com 1, 2 ou 3 pais/filhos sobreviveram mais.

#### Coluna 'Age'

Para analisar a coluna 'Age' irei primeiramente, clusterizá-la, em grupos de acordo com idades parecidas.

In [340]:
def clusterage(age):
    if age <= 17:
      return "Criança"
    elif age <= 30:
      return "Adulto I"
    elif age <= 59:
       return "Adulto II"
    else:
      return "Idoso"

In [341]:
train_age = train.copy()

train_age['Age_Cluster']=train_age['Age'].map(lambda x:clusterage(x))

fig = px.histogram(train_age, x='Age_Cluster', color='Survived', barmode='group')
fig.show()

Proporcionalmente, as Crianças e os Adultos da 2ª faixa foram os que mais sobreviveram.

#### Coluna 'Embarked'

In [342]:
fig = px.histogram(train, x='Embarked', color='Survived', barmode='group')
fig.show()

In [343]:
fig = px.histogram(train, x='Embarked', color='Sex', barmode='group')
fig.show()

As pessoas que embarcaram no porto de Southampton foram as mais prejudicadas.

Podemos perceber que temos um número relativamente maior de homens do que mulheres que embarcaram nesse porto, portanto, isso pode ter sido uma causa para o resultado que achamos anteriormente.

## Dados Nulos

In [344]:
print("Porcentagem de linhas com valores nulos em relação a cada coluna:")
train.isnull().mean()*100

Porcentagem de linhas com valores nulos em relação a cada coluna:


Survived     0.000000
Pclass       0.000000
Sex          0.000000
Age         19.865320
SibSp        0.000000
Parch        0.000000
Fare         0.000000
Embarked     0.224467
dtype: float64

In [345]:
print("Número de linhas com valores nulos por coluna:")
train.isnull().sum()

Número de linhas com valores nulos por coluna:


Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [346]:
print("Porcentagem de linhas com valores nulos em relação a cada coluna:")
test.isnull().mean()*100 

Porcentagem de linhas com valores nulos em relação a cada coluna:


Pclass       0.000000
Sex          0.000000
Age         20.574163
SibSp        0.000000
Parch        0.000000
Fare         0.239234
Embarked     0.000000
dtype: float64

In [347]:
print("Número de linhas com valores nulos por coluna:")
test.isnull().sum()

Número de linhas com valores nulos por coluna:


Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

Irei fazer o tratamento das colunas 'Age' e 'Embarked', através do [KNN Imputer](https://www.notion.so/ufrjanalytica/84d75e8aee944f1b96980f33b613b5b1?v=3b25037759d7450e9e08b0734735d628&p=8a01eede029a47a788baba002f23e69b&pm=c). 

Primeiramente, transformei a coluna 'Sex' que era categórica para uma representação binária.

In [348]:
sex = {'male':1, 'female':0}

train['Sex'] = train['Sex'].map(sex)
test['Sex'] = test['Sex'].map(sex)

train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S


Além disso irei separar a coluna 'Survived' do dataset.

In [349]:
y = train['Survived']
train.drop('Survived', axis=1, inplace=True)

In [350]:
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,Q
1,3,0,47.0,1,0,7.0000,S
2,2,1,62.0,0,0,9.6875,Q
3,3,1,27.0,0,0,8.6625,S
4,3,0,22.0,1,1,12.2875,S


#### KNN Imputer

In [351]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder

#mappin irá guardar os Encoders para cada coluna categórica
mappin = dict()

def encode(data, col):
    # Como "Embarked" possui valores nulos e tem mais de 2 valores possíveis (não binário) 
    # devo aplicar um Encode
    encoder = OrdinalEncoder()
    # O Ordinal Encode apenas trasforma as variáveis categóricas em números de 
    # 0 a (Números de diferentes variáveis)-1. Por exemplo, no caso da coluna 'Embarked',
    # que possui as variávies {C, Q e S}, o C será 0, Q, 1 e o S, 2.

    #todos os valores não nulos da coluna selecionada
    nao_nulos = np.array(data.dropna())
    
    #faço reshape para uma matriz de apenas uma coluna, e as linhas contendo os valores
    nao_nulos = nao_nulos.reshape(-1,1)

    #realizo o encode
    nao_nulos_ordinal = encoder.fit_transform(nao_nulos)
    
    #Coloco os valores "encodados" de volta para coluna, mantendo os NaN
    data[~data.isnull()] = np.squeeze(nao_nulos_ordinal)

    mappin[col] = encoder


def decode(data, col):
    #realizo o mesmo processo do encode
    valores = np.array(data)
    valores = valores.reshape(-1, 1)

    #acesso o Encoder da coluna
    encoder_atual = mappin[col]

    #realizo o encode inverso
    valores = encoder_atual.inverse_transform(valores)

    #coloco os valores de volta na coluna
    data = np.squeeze(valores) 

    return data

# cols = lista com as colunas categóricas do dataset
def imputation(dataset, cols):
    data = dataset.copy()
    
    for variable in cols:
        #para cada variável categórica realizo o encode
        encode(data[variable], variable)

    imputer = KNNImputer()
    
    #realizo o impute através do KNN
    knn = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

    for variable in cols:
          #realizo o round para arrendondar os valores
          knn[variable] = np.round(knn[variable])

          knn[variable] = decode(knn[variable], variable)

    return knn


train = imputation(train, ['Embarked'])
test = imputation(test, ['Embarked'])

print("Número de linhas com valores nulos por coluna:")
train.isnull().sum()

Número de linhas com valores nulos por coluna:


Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [352]:
train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3.0,1.0,22.0,1.0,0.0,7.2500,S
1,1.0,0.0,38.0,1.0,0.0,71.2833,C
2,3.0,0.0,26.0,0.0,0.0,7.9250,S
3,1.0,0.0,35.0,1.0,0.0,53.1000,S
4,3.0,1.0,35.0,0.0,0.0,8.0500,S


Agora que os dados nulos do nosso dataset estão tratados, posso começar a desenvolver o nosso modelo preditivo!

Primeiro irei realizar um One Hot Enconding nas coluna 'Pclass' e 'Embarked'

In [355]:
train = pd.get_dummies(train, columns=['Pclass', 'Embarked'])
test = pd.get_dummies(test, columns=['Pclass', 'Embarked'])

train.head()

,Sex,Age,SibSp,Parch,Fare,Pclass_1.0,Pclass_2.0,Pclass_3.0,Embarked_C,Embarked_Q,Embarked_S
0,1.0,22.0,1.0,0.0,7.2500,0,0,1,0,0,1
1,0.0,38.0,1.0,0.0,71.2833,1,0,0,1,0,0
2,0.0,26.0,0.0,0.0,7.9250,0,0,1,0,0,1
3,0.0,35.0,1.0,0.0,53.1000,1,0,0,0,0,1
4,1.0,35.0,0.0,0.0,8.0500,0,0,1,0,0,1


## Modelo Preditivo

In [356]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score

In [357]:
kfold = StratifiedKFold(5)

In [358]:
model = LogisticRegression(random_state=0)

score = cross_val_score(model, train, y, cv=kfold)

print(score.mean(), score.std())

0.7980038917833155 0.027089167536924006


In [359]:
model = RandomForestClassifier(random_state=0)

score = cross_val_score(model, train, y, cv=kfold)

print(score.mean(), score.std())

0.8126043562864854 0.030440704506328083


In [360]:
model = DecisionTreeClassifier(random_state=0)

score = cross_val_score(model, train, y, cv=kfold)

print(score.mean(), score.std())

0.7811938986880925 0.03369419545061095


In [361]:
model = KNeighborsClassifier()

score = cross_val_score(model, train, y, cv=kfold)

print(score.mean(), score.std())

0.6891595003452389 0.0315757703400601


In [362]:
from sklearn.preprocessing import LabelEncoder

y_encoded = LabelEncoder().fit_transform(y)

model = XGBClassifier()

score = cross_val_score(model, train, y_encoded, cv=kfold)

print(score.mean(), score.std())

0.8125980792166217 0.02503412406361841


Para tentar melhorar o desempenho é possível utilizar algumas clusterizações, como na idade, mas não sei se irá ocorrer uma melhora significante.

In [366]:
model = RandomForestClassifier()
model.fit(train, y)

res = model.predict(test)

test_res = pd.DataFrame()
test_res['PassengerId']=ids
test_res['Survived']=res
test_res['Survived']=test_res['Survived'].astype(np.int64)
test_res.to_csv("submission.csv", index=False)